## 1. 每一张图进行一层推理

In [96]:
import base64
import requests
import api_info

api_key = api_info.api_key
organization = api_info.organization


# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def infer_single_img(image_path, api_key, save_path):
    # Getting the base64 string
    base64_image = encode_image(image_path)

    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "请根据上传的照片，用简洁精炼的语言，描述其中的景色特点。包括但不限于以下方面：\n\
                                    - 自然环境（如山脉、河流、森林、沙滩等)\n\
                                    - 人为元素（如建筑、道路、桥梁等）\n\
                                    - 任何显著的特色或细节（如人群、动物、环境清洁程度等）。",
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            }
        ],
        "max_tokens": 500,
    }

    response = requests.post(
        "https://api.openai.com/v1/chat/completions", headers=headers, json=payload
    )

    with open(save_path, "w", encoding="utf-8-sig") as f:
        f.write(response.json()["choices"][0]["message"]["content"])
    print(f"保存到{save_path}成功！")

## 2. 每一个post的文字+所有图片进行二层推理

In [97]:
import pandas as pd
import os


def get_post_text(index, csv_path):
    df = pd.read_csv(csv_path)
    return df.iloc[index, 1]


def get_post_img_infer(spot_pic_dir):
    # index是文件夹的序号，从0开始
    # base_dir = os.path.join(spot_pic_dir, str(index))
    base_dir = spot_pic_dir
    all_infer = ""
    # 遍历base_dir下的所有的txt文件
    for file in os.listdir(base_dir):
        if file.endswith(".txt"):
            txt_path = os.path.join(base_dir, file)
            with open(txt_path, "r", encoding="utf-8-sig") as f:
                post_text = f.read()
                # print(post_text)
                post_text = post_text + "\n"
                all_infer = all_infer + post_text
    # print(all_infer)
    return all_infer


def get_pic_and_text(index, spot_pic_dir, csv_path):
    all_infer = get_post_img_infer(spot_pic_dir)
    post_text = get_post_text(index, csv_path)
    # 如果post_text不是字符串，则设置为空字符串
    if type(post_text) != str:
        post_text = ""
    all_infer = all_infer + "\n" + post_text
    return all_infer


# spot_pic_dir = r".\data\picture\东西涌"
# csv_path = r".\data\spot_excel\东西涌.csv"
# index = 0
# all_infer = get_pic_and_text(index, spot_pic_dir, csv_path)

In [98]:
from openai import OpenAI


def infer_single_post(all_infer, api_key, organization, save_path, index):
    try:
        df = pd.read_excel(save_path)
    except:
        df = pd.DataFrame(columns=["description"])

    # 如果index已经存在，就不再重复推理
    if index in df.index:
        print(f"index={index}已经存在，不再重复推理！")
        return
    
    client = OpenAI(api_key=api_key, organization=organization)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {
                "role": "system",
                "content": "根据提供的景点描述，请分析并列出以下三个方面的信息：\n\
                            - 优点：列出该景点的主要优点，如自然美景、人文历史、良好的环境保护等。\n\
                            - 缺点：指出该景点可能存在的缺点，例如天气不佳、过于拥挤、交通不便、消费过高等。\n\
                            - 适合的活动：基于景点的特性以及提供的描述，推荐一些适合在此进行的活动，如划船、攀岩、徒步旅行、文化体验等。\n\
                            请分点作答，不要多于200字",
            },
            {
                "role": "user",
                "content": all_infer,
            },
        ],
    )
    # print(response.choices[0].message.content)

    new_row = {"description": response.choices[0].message.content}
    df.loc[len(df)] = new_row
    df.to_excel(save_path, index=False)
    print(f"保存到{save_path}成功！")

## 3. 所有的post的总结进行三层推理

In [99]:
def get_all_post_infer(excel_path):
    df = pd.read_excel(excel_path)
    all_posts_infer = ""
    for i in range(len(df)):
        all_posts_infer = all_posts_infer + df.loc[i, 'description'] + "\n"
    return all_posts_infer

In [100]:
def infer_all_posts(all_posts_infer, api_key, organization, save_path):
    client = OpenAI(api_key=api_key, organization=organization)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {
                "role": "system",
                "content": "根据提供的关于某个景点的大量描述，请帮我总结并简洁地列出该景点的主要特点。请分为以下四部分：\n\
                            - 景色特点：描述该景点的主要特点。如高山，湿地，文化古迹，沙滩等\n\
                            - 优点总结：基于提供的信息，总结出该景点的主要优点。\n\
                            - 缺点总结：从提供的描述中提炼出该景点的主要缺点。\n\
                            - 适合的活动：综合考虑景点的特点，列出几个最适合在此进行的活动项目。\n\
                            请确保每部分的总结全面，便于快速理解该景点的特色，每个部分分点作答。\n",
            },
            {
                "role": "user",
                "content": all_posts_infer,
            },
        ],
    )
    # 判断save_path是否存在，如果不存在则创建，该文件是txt文件
    with open(save_path, "w", encoding="utf-8-sig") as f:
        f.write(response.choices[0].message.content)
    print(f"保存到{save_path}成功！")

# 运行

In [101]:
import os

base_dir = r"./data/picture/东西涌"
text_csv_path = r"./data/spot_excel/东西涌.csv"
each_post_excel_save_path = r"./data/results/东西涌.xlsx"
all_posts_txt_save_path = r"./data/results/东西涌.txt"


def infer_spot(
    base_dir, text_csv_path, each_post_excel_save_path, all_posts_txt_save_path
):
    for i in range(len(os.listdir(base_dir))):
        post_pic_dir = os.path.join(base_dir, str(i))
        pic_num = sum(
            1 for filename in os.listdir(post_pic_dir) if filename.endswith(".jpg")
        )
        for j in range(pic_num):
            post_pic_path = os.path.join(post_pic_dir, str(j) + ".jpg")
            pic_infer_save_path = os.path.join(post_pic_dir, str(j) + ".txt")
            # 如果pic_infer_save_path已经存在，则不需要再次推理
            if os.path.exists(pic_infer_save_path):
                print(f"{pic_infer_save_path}已经存在，不需要再次推理！")
                continue
            infer_single_img(post_pic_path, api_key, pic_infer_save_path)
        all_infer = get_pic_and_text(i, post_pic_dir, text_csv_path)
        infer_single_post(
            all_infer, api_key, organization, each_post_excel_save_path, index=i
        )
    all_posts_infer = get_all_post_infer(each_post_excel_save_path)
    infer_all_posts(all_posts_infer, api_key, organization, all_posts_txt_save_path)


In [102]:
import time
# 如果遇到错误则重新运行该函数
while True:
    try:
        infer_spot(base_dir, text_csv_path, each_post_excel_save_path, all_posts_txt_save_path)
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        time.sleep(10)
        

./data/picture/东西涌\0\0.txt已经存在，不需要再次推理！
./data/picture/东西涌\0\1.txt已经存在，不需要再次推理！
./data/picture/东西涌\0\2.txt已经存在，不需要再次推理！
./data/picture/东西涌\0\3.txt已经存在，不需要再次推理！
./data/picture/东西涌\0\4.txt已经存在，不需要再次推理！
./data/picture/东西涌\0\5.txt已经存在，不需要再次推理！
./data/picture/东西涌\0\6.txt已经存在，不需要再次推理！
./data/picture/东西涌\0\7.txt已经存在，不需要再次推理！
./data/picture/东西涌\0\8.txt已经存在，不需要再次推理！
./data/picture/东西涌\0\9.txt已经存在，不需要再次推理！
./data/picture/东西涌\0\10.txt已经存在，不需要再次推理！
index=0已经存在，不再重复推理！
./data/picture/东西涌\1\0.txt已经存在，不需要再次推理！
./data/picture/东西涌\1\1.txt已经存在，不需要再次推理！
./data/picture/东西涌\1\2.txt已经存在，不需要再次推理！
./data/picture/东西涌\1\3.txt已经存在，不需要再次推理！
./data/picture/东西涌\1\4.txt已经存在，不需要再次推理！
./data/picture/东西涌\1\5.txt已经存在，不需要再次推理！
./data/picture/东西涌\1\6.txt已经存在，不需要再次推理！
./data/picture/东西涌\1\7.txt已经存在，不需要再次推理！
./data/picture/东西涌\1\8.txt已经存在，不需要再次推理！
./data/picture/东西涌\1\9.txt已经存在，不需要再次推理！
./data/picture/东西涌\1\10.txt已经存在，不需要再次推理！
./data/picture/东西涌\1\11.txt已经存在，不需要再次推理！
./data/picture/东西涌\1\12.txt已经存在，不需要再次推理！
./data/picture/东

: 